In [ ]:
import streamlit as st
import requests
import pandas as pd
import xmltodict
import folium
from streamlit_folium import st_folium
import plotly.express as px
import openai
import json
import datetime
import xml.etree.ElementTree as ET

openai.api_key = "MY_OPENAI_KEY"

def parse_with_ai(user_input: str): #input으로 받는 자연어 질의를 LLM을 활용하여 사용자의 요구사항을 파악, dict 형태로 반환함
    prompt = f"""
    사용자의 부동산 검색 요청을 JSON으로 파싱할 수 있도록 구조화해줘.
    반드시 JSON 형식만 출력하고, 추가 설명 금지. 줄바꿈(\n)도 사용하지 말고
    필드는 ["build_year_min", "price_min", "price_max", "floor_space_min", "floor_space_max", "location_do", "location_gu", "location_dong"] 이고
    숫자는 정수(원 단위)로 변환하고, 값이 없으면 null으로 해.

    예시:
    입력: "2006년 이후에 지어졌고 3억~5억, 7평이상 경기도 성남시 아파트"
    출력: {{"build_year_min": 2006, "price_min": 300000000, "price_max": 500000000, "floor_space_min": 7, "floor_space_max": null, "location_do": "경기도", "location_gu": "성남시", "location_dong": null}}

    입력: "{user_input}"
    출력:
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "너는 한국어 부동산 검색 조건을 JSON으로 변환하는 어시스턴트야."},
            {"role": "user", "content": prompt}
        ]
    )
    result = response.choices[0].message["content"]
    #print(result)
    try:
        return json.loads(result)
    except:
        return {}


def get_gu_code(gu): #사용자가 원하는 지역(시/군/구)의 지역번호 찾음
    code=pd.read_csv('/content/drive/MyDrive/에이아이오투오/지역코드.csv')
    gu=gu
    gu_code = code[ (code['name'].str.contains(gu) )]
    gu_code = gu_code['code'].reset_index(drop=True)
    gu_code = str(gu_code[0])[0:5]
    return gu_code


def get_data(gu_code, base_date): #찾아낸 지역번호를 바탕으로 해당 지역에서의 올해 아파트 매매 정보 획득
    url ="http://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade?"
    service_key = "59ff40746ea47eb4443678cb5cd6931e194c8b2157501c64cc46d8a1e0de0a22"
    payload = "LAWD_CD=" + gu_code + "&" + \
              "DEAL_YMD=" + base_date + "&" + \
              "serviceKey=" + service_key + "&"

    res = requests.get(url + payload)
    return res


def get_items(response): #얻은 정보를 xml->딕셔너리 리스트로 변환
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)
    return item_list


def filter_data(df, condition): #앞서 자연어 질의에서 얻은 사용자 요구를 바탕으로 rule-base로 처리하여 요구사항에 맞는 아파트만 필터링
    cond=pd.Series([True]*len(df))
    if condition['build_year_min']:
        cond &= (df['buildYear'].astype(int)>=condition['build_year_min'])
    if condition['price_min']:
        cond &= ((df['dealAmount'].str.replace(',','').astype(int))*10000>=condition['price_min'])
    if condition['price_max']:
        cond &= ((df['dealAmount'].str.replace(',','').astype(int))*10000<=condition['price_max'])
    if condition['floor_space_min']:
        cond &= (df['excluUseAr'].astype(float)>=condition['floor_space_min']*3.306)
    if condition['floor_space_max']:
        cond &= (df['excluUseAr'].astype(float)<=condition['floor_space_max']*3.306)
    if condition['location_dong']:
        cond &= (df['umdNm']==condition['location_dong'])

    return df[cond]


#올해(2025년) 기간 설정
year = [str("%02d" %(y)) for y in range(2025, 2026)]
month = [str("%02d" %(m)) for m in range(1, 9)]
base_date_list = ["%s%s" %(y, m) for y in year for m in month ]


query=input("원하는 부동산 정보를 입력하세요(시/군/구 입력 필수): ")
condition=parse_with_ai(query)
print(condition)

if condition['location_gu'] is None:
    print('원하시는 지역(시/군/구)를 정확히 입력해주세요.')

else:
    gu_code=get_gu_code(condition['location_gu'])

    items_list=[]
    for base_date in base_date_list:
        res = get_data(gu_code, base_date)
        items_list += get_items(res)

    print(len(items_list))
    if len(items_list)==0:
        print('원하시는 아파트 매물이 없습니다.')
    else:
        items = pd.DataFrame(items_list)
        filtered_data=filter_data(items, condition)
        print("요구사항에 맞는 아파트는 다음과 같습니다")
        print(filtered_data.to_markdown())